Data Loading and Preprocessing:

The Iris dataset is loaded and filtered to include only two classes, turning it into a binary classification problem. The data is then standardized.
Model Configurations:

We define the SimpleNN class that allows flexible configuration of the network's hidden layers. It takes a list hidden_layers as input, which defines the number of neurons in each hidden layer.
Training and Evaluation Function:

The train_and_evaluate function initializes a model with the specified hidden layers, trains it using mini-batch gradient descent, and evaluates its accuracy on the test set.
Configurations Comparison:

We specify different configurations of hidden layers and compare them. Each configuration is described by the number of neurons in its hidden layers.
Observations:
By running this code, we can observe how different network architectures (depth and width) affect the model's performance on the binary classification task. Typically, deeper and wider networks can capture more complex patterns but may also require more data and computational power to train effectively. However, for simple datasets like the Iris dataset, simpler models may suffice and even outperform more complex ones due to their lower risk of overfitting.


Compare the configurations a 2-class classification neural network to solve the use case


To compare different configurations of a 2-class classification neural network, we'll implement several models with varying architectures and hyperparameters using the PyTorch framework. We'll use the Iris dataset for simplicity, focusing on two classes to create a binary classification problem.

Setup and Data Preparation
First, we'll load and preprocess the dataset. The Iris dataset will be filtered to include only two classes.

Configurations to Compare
Simple Network:

1 hidden layer with 4 neurons.
Deeper Network:

2 hidden layers with 8 neurons each.
Wider Network:

1 hidden layer with 16 neurons.
More Neurons:

1 hidden layer with 32 neurons.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Load the Iris dataset and filter for binary classification (class 0 and 1)
iris = load_iris()
X = iris.data[iris.target != 2]
y = iris.target[iris.target != 2]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# DataLoader for mini-batch training
batch_size = 16
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model configurations
class SimpleNN(nn.Module):
    def __init__(self, hidden_layers):
        super(SimpleNN, self).__init__()
        layers = []
        input_dim = 4
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            input_dim = hidden_dim
        layers.append(nn.Linear(input_dim, 1))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Function to train and evaluate the model
def train_and_evaluate(hidden_layers):
    model = SimpleNN(hidden_layers)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test_tensor)
        predictions = (test_outputs > 0.5).float()
        accuracy = (predictions == y_test_tensor).float().mean().item()

    return accuracy

# Compare different configurations
configurations = [
    ([4], "1 hidden layer with 4 neurons"),
    ([8, 8], "2 hidden layers with 8 neurons each"),
    ([16], "1 hidden layer with 16 neurons"),
    ([32], "1 hidden layer with 32 neurons")
]

for hidden_layers, description in configurations:
    accuracy = train_and_evaluate(hidden_layers)
    print(f"Configuration: {description}, Test Accuracy: {accuracy:.4f}")


Configuration: 1 hidden layer with 4 neurons, Test Accuracy: 1.0000
Configuration: 2 hidden layers with 8 neurons each, Test Accuracy: 1.0000
Configuration: 1 hidden layer with 16 neurons, Test Accuracy: 1.0000
Configuration: 1 hidden layer with 32 neurons, Test Accuracy: 1.0000
